In [1]:
import warnings
import sys
import multiprocessing
import networkx as nx
from sklearn.model_selection import ParameterGrid
from matplotlib import pyplot as plt
import time
import os

from ABM_Func import *

In [2]:
from input_parameters import *

In [3]:
# # New Parallelization Package
# from mpi4py import MPI
# #Retrieves MPI environment
# comm = MPI.COMM_WORLD
# #set variable "size" as the total number of MPI processes
# size = comm.Get_size()
# #print(size)
# #set variable "rank" as the specific MPI rank on each MPI process
# rank = comm.Get_rank()

In [4]:
# Exception graph construction for single agents
if num_agents == 1:

    graphs = [nx.Graph() for _ in range(num_graphs)]
    for graph in graphs:
        graph.add_node(1)

    graph_labels = dict(zip(graphs, range(len(graphs)))) 

# Create graph/graphs for multiple agents
elif num_agents > 1:

    graphs = []     
    if len(edges) != 1:
        for edge in edges:
            graphs.append([nx.gnm_random_graph(num_agents, edge) for _ in range(num_graphs)])
        graphs = [graph[i] for graph in graphs for i in range(len(graph))] 
        
    graph_labels = dict(zip(graphs, range(len(graphs)))) 

In [5]:
# Create Parameter Grid
parameter_grid = {'landscape': landscapes, 
                'num_agents': [num_agents], 
                'num_searches': [num_searches], 
                'graph': graphs, 
                'memory': memory,
                'distance_costs': distance_costs,
                'myopia': [myopia],
                'sync': [True]}

# Create Parameter Space
parameter_combos = list(ParameterGrid(parameter_grid))
parameter_combos = [a.copy() for a in parameter_combos for _ in range(num_sims)]

# Labels Param_Combos with IDs for Identification
for i, pcombo in enumerate(parameter_combos):
    # Add id key
    pcombo.update({'id': i})
    # Add graph_label key
    pcombo.update({'graph_label': graph_labels[pcombo['graph']]})

#Shuffle so batches aren't unbalanced
random.shuffle(parameter_combos)

In [6]:
size = 10

In [7]:
## Split Parameter Space for Distribution
split_seeds = np.array_split(parameter_combos, size, axis=0)

In [8]:
p_combo = split_seeds[0][0]

In [36]:
sim_hist = run_model(p_combo)

Population initialized!
Sim Complete!


In [46]:
sim_hist.landscape.tofile(f'{sim_hist.id_}_landscape.csv',sep=',')

In [45]:
sim_hist.id_

0

In [40]:
sim_hist.landscape

array([[0.51178171, 0.57399155, 0.65174406, ..., 0.40474852, 0.39161725,
        0.43528239],
       [0.48344098, 0.54215268, 0.62063948, ..., 0.45052032, 0.4487058 ,
        0.49098279],
       [0.53519413, 0.56768498, 0.60528794, ..., 0.4950774 , 0.51058813,
        0.54146068],
       ...,
       [0.44970332, 0.46146217, 0.51315424, ..., 0.58383594, 0.58624565,
        0.59152567],
       [0.47373236, 0.48893879, 0.52219909, ..., 0.62713574, 0.63864553,
        0.61578942],
       [0.50788396, 0.51044145, 0.49120802, ..., 0.62710583, 0.65764109,
        0.62076374]])